# Mental Health Assessment Tool - Project Summary

## 1. Introduction

This project builds a Mental Health Assessment Assistant using Retrieval-Augmented Generation (RAG) techniques. The assistant aims to:

- Provide relevant mental health advice.

- Retrieve verified information from a curated dataset.

- Generate responses using a lightweight LLM (Llama 3) via API.

The primary focus is efficient local vector search combined with remote model inference.

## 2. Dataset

The data sources included multiple CSV files related to mental health forums, assessments, and therapy discussions. These were merged into a single large file:

- Merged dataset: dataset/merged_posts.txt

- Content: User posts, Q&A, therapy discussions, mental health experiences.

The dataset was cleaned and prepared for text chunking.

## 3. Vectorstore Building

We created a persistent vectorstore using ChromaDB and SentenceTransformer (all-MiniLM-L6-v2):

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch

# Load model with GPU acceleration
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# Connect to ChromaDB persistent storage
chroma_client = chromadb.PersistentClient(path="vectorstore")
collection = chroma_client.get_or_create_collection("mental_health")

# Chunk dataset and embed using batch processing
# Store embeddings and text chunks into the ChromaDB collection

## 4. Retrieval Mechanism

For every user query, we:

- Encode the query.

- Perform vector similarity search in ChromaDB.

- Retrieve the top K=3 most relevant chunks.

This ensures the model gets rich, specific context.

In [ ]:
def retrieve_similar_chunks(query, top_k=3):
    query_embedding = model.encode([query]).tolist()
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=["documents"]
    )
    return results["documents"][0]

## 5. LLM Integration

Initially, we used a locally running Llama 3 model, but due to resource constraints, we switched to a remote API-based model hosting (like Together.ai or Ollama local server).

In [ ]:
import requests

def get_llama_response(prompt, context=""):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3",
            "prompt": f"Context: {context}\n\nQuestion: {prompt}\nAnswer:",
            "stream": False
        }
    )
    return response.json()["response"].strip()

## 6. Chatbot Structure

We combined retrieval and generation into a simple chatbot loop:

In [ ]:
from vectorstore import retrieve_similar_chunks
from llm_integration import get_llama_response

def chat():
    while True:
        query = input("You: ")
        if query.lower() == 'quit':
            break
        context_chunks = retrieve_similar_chunks(query)
        context = "\n".join(context_chunks)
        response = get_llama_response(prompt=query, context=context)
        print(f"\nAssistant: {response}\n")

if __name__ == "__main__":
    chat()

This system is lightweight, modular, and easy to expand.

## 7. Deployment Info

- Vectorstore is stored in the vectorstore/ folder.

- LLM API must be active at http://localhost:11434.

- Simply run:

    python chatbot.py

- Chatbot will start interacting based on RAG outputs.



## 8. Future Scope

### Short-Term Goals

- Conversational Memory
  
- Mood Tracing

- Add timers to restrict chatbot session activity.

- Implement dynamic multi-turn conversations.

- Design a user mental health assessment flow (objective + subjective).

- Build a streamlined frontend (already under planning).

### Long-Term Enhancements

- Personalize responses based on user profiles.

- Provide therapy-style guidance using dataset knowledge.

- Integrate with larger LLMs or finetuned domain models.

- Host the backend (API + vectorstore) on cloud for scalability.



## Links

GitHub Repository - https://github.com/shutupatul/manasooth-bot